In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import glob
import sys
import tensorflow as tf
from datetime import datetime

In [2]:
# Helper functions, DO NOT modify this

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return scipy.misc.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data, DO NOT change these

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    y_one_hot[y] = 1
    return y_one_hot.T

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)

    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)# Load the data

In [4]:
# Load the data
data_root_path = '/home/ec2-user/Neural_Network/Neural_Network/cifar10-hw1/'
X_train, y_train = get_train_data(data_root_path) # this may take a few minutes
X_test = get_images(data_root_path + 'test').T

print('Data loading done')
indexs = np.random.choice(50000, 5000, replace=False)
X_validation = X_train[:,indexs].T
y_validation = y_train[indexs]
X_train = np.delete(X_train,indexs, axis = 1).T
y_train = np.delete(y_train,indexs)

{'dog': 5, 'automobile': 1, 'airplane': 0, 'horse': 7, 'ship': 8, 'truck': 9, 'cat': 3, 'deer': 4, 'bird': 2, 'frog': 6}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
print(X_validation.shape)
print(X_test.shape)
arr = np.arange(X_train.shape[0])
np.random.shuffle(arr)
print(arr)

(5000, 3072)
(10000, 3072)
[17639 36412  5517 ..., 32275 20723 27192]


In [6]:
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        m = X.shape[0]
        start_index = np.random.randint(0, m - batch_size)
        X_batch = X[start_index:(start_index + batch_size), :]
        y_batch = y[start_index:(start_index + batch_size)]
        
        return X_batch, y_batch

In [7]:
logdir = 'tf_logs/summary7'
iteration = 20000
batch_size = 128
learning_rate = 0.01

In [8]:
# try use a function to create a layer
# def create_conv(filters, input_layer):
#     conv = tf.layers.conv2d(inputs=input_layer, filters=filters, kernel_size=3, strides=1, 
#                                     padding = 'SAME', activation = tf.nn.relu, name='conv')
#     pool = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2, padding='SAME', name='pool')
#     return pool

In [9]:
def save_variable(var, name):
    with tf.name_scope(name) as scope:
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        stddev = tf.sqrt(tf.reduce_mean(var - mean))
        tf.summary.scalar('stddev', stddev)
        tf.summary.histogram('histgram', var)

In [10]:
with tf.device('/gpu:0'):
    with tf.name_scope('input') as scope:
        X = tf.placeholder(tf.float32, shape = (None, 3072))
        y = tf.placeholder(tf.int32, shape = (None))
        dropout_rate = tf.placeholder(tf.float32, shape=())
        training = tf.placeholder(tf.bool)
        input_layer = tf.reshape(X, [-1, 32, 32, 3])

    with tf.name_scope('conv1') as scope:
        conv1 = tf.layers.conv2d(inputs=input_layer, filters=64, kernel_size=3, strides=1, 
                                padding = 'SAME', activation = tf.nn.relu, name='conv1')
#         weight1 = tf.contrib.layers.flatten(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1'))
        weight1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv1')[0]
        batch_norm1 = tf.layers.batch_normalization(conv1, axis = 1, name='batch_norm1')
        dropout1 = tf.layers.dropout(batch_norm1,rate=0.1, training=training, name='dropout1')
        pool1 = tf.layers.max_pooling2d(inputs=dropout1, pool_size=2, strides=2, padding='SAME', name='pool1')

    with tf.name_scope('conv2') as scope:
        conv2 = tf.layers.conv2d(inputs=pool1, filters=128, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv2')
        weight2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv2')[0]
        batch_norm2 = tf.layers.batch_normalization(conv2, axis = 1, name='batch_norm2')
        dropout2 = tf.layers.dropout(batch_norm2,rate=0.2, training=training, name='dropout2')
        pool2 = tf.layers.max_pooling2d(inputs=dropout2, pool_size=2, strides=2, padding='SAME', name='pool2')  
        
    with tf.name_scope('conv3') as scope:
        conv3 = tf.layers.conv2d(inputs=pool2, filters=128, kernel_size=3, strides=1, 
                            padding = 'SAME', activation = tf.nn.relu, name='conv3')
        weight3 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'conv3')[0]
        batch_norm3 = tf.layers.batch_normalization(conv3, axis = 1, name='batch_norm3')
        dropout3 = tf.layers.dropout(batch_norm3,rate=0.2, training=training, name='dropout3')
        pool3 = tf.layers.max_pooling2d(inputs=dropout3, pool_size=2, strides=2, padding='SAME', name='pool3')

    with tf.name_scope('fc') as scope:
        pool3_flat = tf.reshape(pool3, [-1, 4*4*128], name='pool3_flat')
        dropout_fc1 = tf.layers.dropout(pool3_flat, rate=0.2, training=training, name='dropout_fc1')
        
        dense1 = tf.layers.dense(dropout_fc1, units = 8*128, activation = tf.nn.relu, name='dense1')
        batch_norm4 = tf.layers.batch_normalization(dense1, axis = 1, name='batch_norm4')
        dropout_fc2 = tf.layers.dropout(batch_norm4, rate=0.2, training=training, name='dropout_fc2')
        
        dense2 = tf.layers.dense(dropout_fc2, units = 128, activation = tf.nn.relu, name='dense2') 
        batch_norm5 = tf.layers.batch_normalization(dense2, axis = 1, name='batch_norm5')
        dropout_fc3 = tf.layers.dropout(batch_norm5, rate=0.2, training=training, name='dropout_fc3')


    with tf.name_scope('logits') as scope:
        logits = tf.layers.dense(dropout_fc3, units = 10, name='logits')
        weight_logits = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'logits')[0]

    with tf.name_scope('loss') as scope:
        softmax = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name = 'softmax')
        loss = tf.reduce_mean(softmax) 

    with tf.name_scope('train') as scope:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

    with tf.name_scope('eval') as scope:
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
     
    with tf.name_scope('summary') as scope:
        save_variable(weight1,'weight1')
        save_variable(weight2,'weight2')
        save_variable(weight3,'weight3')
        save_variable(weight_logits,'weight_logits')
        tf.summary.scalar('loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(logdir + '/train', tf.get_default_graph())
        test_writer = tf.summary.FileWriter(logdir + '/test', tf.get_default_graph())

    with tf.name_scope('init') as scope:
        init = tf.global_variables_initializer()
    

In [11]:
print(weight_logits)

<tf.Variable 'logits/kernel:0' shape=(128, 10) dtype=float32_ref>


In [12]:
def train(X_train, y_train, X_validation, y_validation, iteration, X_test):

    config = tf.ConfigProto()
    config.allow_soft_placement = True
    
    with tf.Session(config=config) as sess:
        sess.run(init)
        for step in range(iteration):
            X_batch, y_batch = get_batch(X_train, y_train, batch_size)
            sess.run(optimizer, feed_dict={X: X_batch, y:y_batch, dropout_rate:0.3, training:True})
                
            if step % 100 == 0:
                summary, loss_, acc = sess.run([merged, loss, accuracy],
                                               feed_dict={X: X_validation, y:y_validation, dropout_rate:0.3, training:False})
                test_writer.add_summary(summary, step)
                print('###################################')
                print('validation! after '+ str(step) + ' iterations' + 
                              ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
                        
                summary, loss_, acc = sess.run([merged, loss, accuracy], 
                                               feed_dict={X: X_batch, y:y_batch,dropout_rate:0.3, training:False})
                train_writer.add_summary(summary, step)
                        
                print('training! after '+ str(step) + ' iterations' + 
                          ' the loss is ' + str(loss_) + ', the accuracy is ' + str(acc))
            
        y_labels = logits.eval(feed_dict={X: X_test, dropout_rate:0.3, training:False})
    return y_labels.T

In [13]:
y_predicted = train(X_train, y_train, X_validation, y_validation, iteration, X_test)

###################################
validation! after 0 iterations the loss is 2.30343, the accuracy is 0.0954
training! after 0 iterations the loss is 2.30063, the accuracy is 0.101562
###################################
validation! after 100 iterations the loss is 2.2835, the accuracy is 0.173
training! after 100 iterations the loss is 2.27213, the accuracy is 0.257812
###################################
validation! after 200 iterations the loss is 2.24183, the accuracy is 0.2078
training! after 200 iterations the loss is 2.22378, the accuracy is 0.273438
###################################
validation! after 300 iterations the loss is 2.16121, the accuracy is 0.2436
training! after 300 iterations the loss is 2.17708, the accuracy is 0.289062
###################################
validation! after 400 iterations the loss is 2.10077, the accuracy is 0.2786
training! after 400 iterations the loss is 2.07269, the accuracy is 0.335938
###################################
validation! after 50

###################################
validation! after 4300 iterations the loss is 1.36201, the accuracy is 0.5112
training! after 4300 iterations the loss is 1.33915, the accuracy is 0.46875
###################################
validation! after 4400 iterations the loss is 1.34868, the accuracy is 0.513
training! after 4400 iterations the loss is 1.25636, the accuracy is 0.546875
###################################
validation! after 4500 iterations the loss is 1.36881, the accuracy is 0.4912
training! after 4500 iterations the loss is 1.30371, the accuracy is 0.5625
###################################
validation! after 4600 iterations the loss is 1.37232, the accuracy is 0.491
training! after 4600 iterations the loss is 1.10571, the accuracy is 0.5625
###################################
validation! after 4700 iterations the loss is 1.38724, the accuracy is 0.5008
training! after 4700 iterations the loss is 1.24034, the accuracy is 0.523438
###################################
validation!

###################################
validation! after 8600 iterations the loss is 1.03891, the accuracy is 0.6388
training! after 8600 iterations the loss is 0.973095, the accuracy is 0.648438
###################################
validation! after 8700 iterations the loss is 1.06327, the accuracy is 0.6256
training! after 8700 iterations the loss is 0.97481, the accuracy is 0.671875
###################################
validation! after 8800 iterations the loss is 1.05392, the accuracy is 0.6218
training! after 8800 iterations the loss is 0.872382, the accuracy is 0.734375
###################################
validation! after 8900 iterations the loss is 1.01334, the accuracy is 0.6476
training! after 8900 iterations the loss is 0.89324, the accuracy is 0.695312
###################################
validation! after 9000 iterations the loss is 1.01904, the accuracy is 0.641
training! after 9000 iterations the loss is 0.832019, the accuracy is 0.6875
###################################
vali

###################################
validation! after 12900 iterations the loss is 0.879704, the accuracy is 0.69
training! after 12900 iterations the loss is 0.49841, the accuracy is 0.804688
###################################
validation! after 13000 iterations the loss is 0.863437, the accuracy is 0.6938
training! after 13000 iterations the loss is 0.581153, the accuracy is 0.78125
###################################
validation! after 13100 iterations the loss is 0.902848, the accuracy is 0.68
training! after 13100 iterations the loss is 0.646044, the accuracy is 0.789062
###################################
validation! after 13200 iterations the loss is 0.848065, the accuracy is 0.7032
training! after 13200 iterations the loss is 0.677877, the accuracy is 0.757812
###################################
validation! after 13300 iterations the loss is 0.866939, the accuracy is 0.6934
training! after 13300 iterations the loss is 0.543812, the accuracy is 0.835938
##########################

###################################
validation! after 17200 iterations the loss is 0.79091, the accuracy is 0.7262
training! after 17200 iterations the loss is 0.491445, the accuracy is 0.789062
###################################
validation! after 17300 iterations the loss is 0.789205, the accuracy is 0.7288
training! after 17300 iterations the loss is 0.622443, the accuracy is 0.804688
###################################
validation! after 17400 iterations the loss is 0.782237, the accuracy is 0.7334
training! after 17400 iterations the loss is 0.511736, the accuracy is 0.804688
###################################
validation! after 17500 iterations the loss is 0.785856, the accuracy is 0.727
training! after 17500 iterations the loss is 0.637227, the accuracy is 0.78125
###################################
validation! after 17600 iterations the loss is 0.794403, the accuracy is 0.727
training! after 17600 iterations the loss is 0.486058, the accuracy is 0.835938
########################

In [14]:
save_predictions('ans1-yz3065', y_predicted)

In [15]:
# test if your numpy file has been saved correctly
loaded_y = np.load('ans1-yz3065.npy')
print(loaded_y.shape)
loaded_y[:10]

(10, 10000)


array([[ -2.11895514,   5.37682199,   3.69640088, ...,  -2.88676643,
          0.7375437 ,  -2.25844049],
       [ -1.8277669 ,   7.39285707,  -4.63892508, ...,  -5.2998457 ,
          1.91594195,  -4.43879604],
       [ -1.09438086,  -2.52297997,   1.58614469, ...,   2.10775352,
          0.562217  ,   2.13540459],
       ..., 
       [ -0.75654238,  -5.59014511,  -1.42161751, ...,   3.26914692,
         -0.34631056,  11.20080853],
       [  1.13695455,  10.96027851,   1.32668006, ...,  -0.9483766 ,
         -3.46490669,  -5.88969469],
       [ -1.40644372,   2.9334054 ,  -3.40348506, ...,  -3.89431953,
         -1.54861367,  -3.46016812]], dtype=float32)